In [1]:
from piece import piece

piece.start()


In [84]:
piece.stop()

In [ ]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.generative import *
from soundmining_tools.modular.instrument import NodeId

piece.reset()
piece.synth_player.start()

reverb_effect = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .stereo_input() \
    .stereo_free_reverb(piece.control_instruments.static_control(1), mix=0.99, room=0.99) 

reverb_effect.play(0, 30)


clean_effect_note = piece.synth_player.note(NodeId.EFFECT) \
    .stereo_input()
    
clean_effect_note.send_to_synth_note(reverb_effect, 0, 30)

delay_effect_note = piece.synth_player.note(NodeId.EFFECT) \
    .input_from_note(clean_effect_note) \
    .stereo_comb(piece.control_instruments.static_control(1), 0.5, 3) \
    .send_to_synth_note(reverb_effect, 0, 30)


def pan_line(distance: float) -> tuple[float, float]:
        random_distance = distance * random_range(0.8, 1.2)
        starting_point = random_range(-0.75, 0.75)
        direction = random.choice([-1.0, 1.0])
        desired_endpoint = (starting_point + random_distance) * direction
        endpoint = max(min(desired_endpoint, 1.0), -1.0)
        return (starting_point, endpoint)

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        amp = patch_arguments.amp

        match patch_arguments.octave:
            case 3:
                xline_duration = random_range(0.85, 1.50)
                pan_position = random_range(-0.95, 0.95)
                piece.synth_player.note() \
                    .white_noise(piece.control_instruments.xline_control(amp, 0.000001)) \
                    .pan(piece.control_instruments.static_control(pan_position)) \
                    .send_to_synth_note(clean_effect_note, patch_arguments.start, xline_duration)                
            case 4:
                sine_duration = random_range(0.75, 1.25)
                start_pan, end_pan = pan_line(0.5)
                piece.synth_player.note() \
                    .white_noise(piece.control_instruments.sine_control(0, amp)) \
                    .pan(piece.control_instruments.line_control(start_pan, end_pan)) \
                    .send_to_synth_note(clean_effect_note, patch_arguments.start, sine_duration)                    


my_handler = MyHandler(piece.supercollider_client)

piece.receiver.set_note_handler(my_handler)
        

In [10]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.generative import *
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools import note
from soundmining_tools import spectrum
from soundmining_tools.sieve import *
import math
from typing import Callable
from soundmining_tools.sequencer import Sequencer, SequenceNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipycanvas import Canvas

piece.reset()
piece.synth_player.start()

reverb_effect_middle_high = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .stereo_input() \
    .stereo_free_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), mix=0.9, room=0.90) 

reverb_effect_middle_high.play(0, 120)

reverb_effect_middle = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .stereo_input() \
    .stereo_free_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), mix=0.7, room=0.70) 

reverb_effect_middle.play(0, 120)



effect_low = piece.synth_player.note(NodeId.EFFECT) \
    .stereo_input() \
    
reverb_effect_low = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .input_from_note(effect_low) \
    .stereo_volume(piece.control_instruments.static_control(0.2)) \
    .stereo_hall_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), rt60=5)
reverb_effect_low.play(0, 120)

clean_effect_low = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .input_from_note(effect_low) \
    .stereo_volume(piece.control_instruments.static_control(0.8))
clean_effect_low.play(0, 120)


effect_high = piece.synth_player.note(NodeId.EFFECT) \
    .stereo_input() \
    
reverb_effect_high = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .input_from_note(effect_high) \
    .stereo_volume(piece.control_instruments.static_control(0.2)) \
    .stereo_hall_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), rt60=8)
reverb_effect_high.play(0, 120)

clean_effect_high = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .input_from_note(effect_high) \
    .stereo_volume(piece.control_instruments.static_control(0.8))
clean_effect_high.play(0, 120)


def pan_line(distance: float, ranges: list[tuple[float, float]] = [(-0.99, 0.99)]) -> tuple[float, float]:
        range_start, range_end = random.choices(ranges, k=1)[0]
        random_distance = distance * random_range(0.8, 1.2)
        starting_point = random_range(range_start, range_end)
        direction = random.choice([-1.0, 1.0])
        desired_endpoint = (starting_point + random_distance) * direction
        endpoint = max(min(desired_endpoint, range_end), range_start)
        return (starting_point, endpoint)

def pan_point(ranges: list[tuple[float, float]] = [(-0.99, 0.99)]) -> float:
        range_start, range_end = random.choices(ranges, k=1)[0]
        return random_range(range_start, range_end)


def make_spectrums(fundamental_note: str, first_partial_note: str, scale_size: int, octave: int) -> list[list[float]]:
    fundamental = note.note_to_hertz(fundamental_note)
    first_partial = note.note_to_hertz(first_partial_note)
    fact = spectrum.make_fact(fundamental, first_partial)

    fundamental_cents = note.hertz_to_cents(fundamental)
    octave_cents = note.hertz_to_cents(first_partial) - fundamental_cents
    low_cents = fundamental_cents + (octave_cents * octave)
    note_cents = octave_cents / scale_size
    scale = [note.cents_to_hertz(low_cents + (note_cents * i)) for i in range(11)]
    return [spectrum.make_spectrum(note, fact, 15) for note in scale]

middle_spectrums1 = make_spectrums("d3", "c4", 5, 0)
middle_spectrums2 = make_spectrums("c3", "ciss4", 5, 0)
middle_high_spectrums1 = make_spectrums("d3", "c4", 5, 1)
middle_high_spectrums2 = make_spectrums("c3", "ciss4", 5, 1)
low_spectrums1 = make_spectrums("d3", "c4", 5, -2)
low_spectrums2 = make_spectrums("c3", "ciss4", 5, -2)
high_spectrums1 = make_spectrums("d3", "c4", 5, 2)
high_spectrums2 = make_spectrums("c3", "ciss4", 5, 2)

sieve1 = SimpleSieve(2, 0)
sieve2 = SimpleSieve(3, 1)
amp_values1 = lambda i: 1.0 / (i + 1.0)
amp_values2 = lambda i: 1.0 / ((i +1) * math.pi)
amp_values3 = lambda i: 4.0 / math.pow((i + 1.0) * math.pi, 2)
ring_times1 = [1.0 / (i + 1.0) for i in range(15)]
ring_times2 = [random_range(0.2, 0.9) for i in range(15)]
ring_times3 = [random_range(0.05, 0.25) for i in range(15)]

def make_amp_values(sieve: Sieve, value_func: Callable[[int], float]) -> list[float]:
    return [value_func(i) if sieve.is_sieve(i) else 0.0 for i in range(15)]

def choose_high_note(start_time: float, spectrums: list[list[float]], color: str, track: str) -> SequenceNote:
    amp = random_range(0.15, 0.85) * 0.01
    sieve = random.choices([sieve1, sieve2], weights=[0.4, 0.6], k=1)[0]     
    amp_values = make_amp_values(sieve, amp_values1)
    ring_times = ring_times2
    note = random_int_range(0, 10)
    note_spec = spectrums[note]
    duration = random_range(13, 21)
    start_pan, end_pan = pan_line(0.5, [(-0.99, -0.66), (0.66, 0.99)])
    amp_time = random_range(0.55, 0.80)
    piece.synth_player.note() \
        .white_noise(piece.control_instruments.two_block_control((0, amp, 0), (1 - amp_time, amp_time), (0, 0))) \
        .bank_of_resonators(note_spec, amp_values, ring_times) \
        .mono_high_pass_filter(piece.control_instruments.static_control(2000)) \
        .pan(piece.control_instruments.line_control(start_pan, end_pan)) \
        .send_to_synth_note(effect_high, start_time, duration)
        #.play(start_time, duration)
    return SequenceNote(start=start_time, track=track, duration=duration, note=note, peak=amp_time, color=color)

def choose_low_note(start_time: float, spectrums: list[list[float]], color: str, track: str) -> SequenceNote:
    amp = random_range(0.15, 0.85) * 0.01
    sieve = random.choices([sieve1, sieve2], weights=[0.4, 0.6], k=1)[0]     
    amp_values = make_amp_values(sieve, amp_values1)
    ring_times = ring_times2
    note = random_int_range(0, 10)
    note_spec = spectrums[note]
    duration = random_range(5, 8)
    start_pan, end_pan = pan_line(0.5, [(-0.66, 0.66)])
    amp_time = random_range(0.55, 0.80)
    piece.synth_player.note() \
        .white_noise(piece.control_instruments.two_block_control((0, amp, 0), (amp_time, 1 - amp_time), (0, 0))) \
        .bank_of_resonators(note_spec, amp_values, ring_times) \
        .mono_low_pass_filter(piece.control_instruments.static_control(500)) \
        .pan(piece.control_instruments.line_control(start_pan, end_pan)) \
        .send_to_synth_note(effect_low, start_time, duration)
        #.send_to_synth_note(reverb_effect_low, start_time, duration)
        #.play(start_time, duration)
    return SequenceNote(start=start_time, track=track, duration=duration, note=note, peak=amp_time, color=color)

def choose_middle_high_note(start_time: float, spectrums: list[list[float]], color: str, track: str) -> SequenceNote:
    amp = random_range(0.15, 0.75) * 0.05
    sieve = random.choices([sieve1, sieve2], weights=[0.2, 0.8], k=1)[0]     
    amp_values = make_amp_values(sieve, amp_values1)
    ring_times = ring_times2
    note = random_int_range(0, 10)
    note_spec = spectrums[note]
    note_type = random.choices([0, 1], weights=[0.80, 0.2], k=1)[0]
    match note_type:
        case 0:
            xline_duration = random_range(0.25, 0.75)        
            pan_position = pan_point([(-0.99, -0.66), (0.66, 0.99)])
            piece.synth_player.note() \
                .white_noise(piece.control_instruments.xline_control(amp, 0.000001)) \
                .bank_of_resonators(note_spec, amp_values, ring_times) \
                .mono_low_pass_filter(piece.control_instruments.static_control(600)) \
                .mono_high_pass_filter(piece.control_instruments.static_control(200)) \
                .pan(piece.control_instruments.static_control(pan_position)) \
                .send_to_synth_note(reverb_effect_middle_high, start_time, xline_duration)
                #.play(start_time, xline_duration)
            return SequenceNote(start=start_time, track=track, duration=xline_duration, note=note, peak=0.000001, color=color)
        case 1:
            sine_duration = random_range(0.15, 0.40)            
            start_pan, end_pan = pan_line(0.3, [(-0.99, -0.66), (0.66, 0.99)])
            piece.synth_player.note() \
                .white_noise(piece.control_instruments.sine_control(0, amp)) \
                .bank_of_resonators(note_spec, amp_values, ring_times) \
                .mono_low_pass_filter(piece.control_instruments.static_control(600)) \
                .mono_high_pass_filter(piece.control_instruments.static_control(300)) \
                .pan(piece.control_instruments.line_control(start_pan, end_pan)) \
                .send_to_synth_note(reverb_effect_middle_high, start_time, sine_duration)
                #.play(start_time, sine_duration)
            return SequenceNote(start=start_time, track=track, duration=sine_duration, note=note, peak=0.5, color=color)

def choose_middle_note(start_time: float, spectrums: list[list[float]], color: str, track: str) -> SequenceNote:
    amp = random_range(0.15, 0.85) * 0.02
    sieve = random.choices([sieve1, sieve2], weights=[0.4, 0.6], k=1)[0]     
    amp_values = make_amp_values(sieve, amp_values1)
    ring_times = ring_times2
    note = random_int_range(0, 10)
    note_spec = spectrums[note]
    note_type = random.choices([0, 1], weights=[0.80, 0.2], k=1)[0]
    match note_type:
        case 0:
            xline_duration = random_range(0.85, 1.50)
            pan_position = pan_point([(-0.66, 0.66)])
            piece.synth_player.note() \
                .white_noise(piece.control_instruments.xline_control(amp, 0.000001)) \
                .bank_of_resonators(note_spec, amp_values, ring_times) \
                .pan(piece.control_instruments.static_control(pan_position)) \
                .send_to_synth_note(reverb_effect_middle, start_time, xline_duration)
                #.play(start_time, xline_duration)
            return SequenceNote(start=start_time, track=track, duration=xline_duration, note=note, peak=0.000001, color=color)
        case 1:
            sine_duration = random_range(0.75, 1.25)
            start_pan, end_pan = pan_line(0.5, [(-0.66, 0.66)])
            piece.synth_player.note() \
                .white_noise(piece.control_instruments.sine_control(0, amp)) \
                .bank_of_resonators(note_spec, amp_values, ring_times) \
                .pan(piece.control_instruments.line_control(start_pan, end_pan)) \
                .send_to_synth_note(reverb_effect_middle, start_time, sine_duration)
                #.play(start_time, sine_duration)
            return SequenceNote(start=start_time, track=track, duration=sine_duration, note=note, peak=0.5, color=color)

def choose_low_melody(start_time: float, spectrums: list[list[float]], color: str) -> list[SequenceNote]:
    sequence_notes = []
    nr_of_low_notes = random_int_range(1, 3)                
    current_low_time = start_time
    for i in range(nr_of_low_notes):
        sequence_notes.append(choose_low_note(current_low_time, spectrums, color, "Low"))
        current_low_time = current_low_time + random_range(2, 3)
    return sequence_notes

def choose_middle_high_melody(start_time: float, spectrums: list[list[float]], color: str) -> list[SequenceNote]:
    sequence_notes = []
    nr_of_middle_notes = random_int_range(5, 8)
    current_middle_time = start_time
    for i in range(nr_of_middle_notes):
        sequence_notes.append(choose_middle_high_note(current_middle_time, spectrums, color, "Middle High"))
        current_middle_time = current_middle_time + random_range(0.05, 0.45)
    return sequence_notes

def choose_middle_melody(start_time: float, spectrums: list[list[float]], color: str) -> list[SequenceNote]:
    sequence_notes = []
    nr_of_middle_notes = random_int_range(1, 5)
    current_middle_time = start_time
    for i in range(nr_of_middle_notes):
        sequence_notes.append(choose_middle_note(current_middle_time, spectrums, color, "Middle"))
        current_middle_time = current_middle_time + random_range(0.1, 1.5)
    return sequence_notes

def choose_high_melody(start_time: float, spectrums: list[list[float]], color: str) -> list[SequenceNote]:
    sequence_notes = []
    nr_of_high_notes = random_int_range(1, 2)
    current_high_time = start_time
    for i in range(nr_of_high_notes):
        sequence_notes.append(choose_high_note(current_high_time, spectrums, color, "High"))
        current_high_time = current_high_time + random_range(3, 5)
    return sequence_notes

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:        
        match patch_arguments.octave:
            case 2:
                choose_low_melody(patch_arguments.start, low_spectrums1, "black")
            case 3:
                choose_middle_high_melody(patch_arguments.start, middle_high_spectrums1, "black")
            case 4:
                choose_middle_melody(patch_arguments.start, middle_spectrums1, "black")
            case 5:
                choose_high_melody(patch_arguments.start, high_spectrums1, "black")
           
#my_handler = MyHandler(piece.supercollider_client)

#piece.receiver.set_note_handler(my_handler)

middle_high_times = [8, 8, 8, 8, 8, 8, 8, 8]
middle_high_spectrums = [2, 1, 1, 1, 1, 1, 1, 1]

def middle_high_step_handler(i: int, time: float) -> list[SequenceNote]:
    match middle_high_spectrums[i]:
        case 1:
            return choose_middle_high_melody(time, middle_high_spectrums1, "black")
        case 2:
            return choose_middle_high_melody(time, middle_high_spectrums2, "red")

middle_high_sequencer = Sequencer(len(middle_high_times)) \
    .add_step_handler(middle_high_step_handler) \
    .start_time_handler(lambda time: time + random_range(3, 5)) \
    .next_time_handler(lambda i: middle_high_times[i] * random_range(0.85, 1.15))

high_times = [21, 21, 21, 21]
high_spectrums = [1, 1, 1, 2]

def high_step_handler(i: int, time: float) -> list[SequenceNote]:
    match high_spectrums[i]:
        case 1:
            return choose_high_melody(time, high_spectrums1, "black")
        case 2:
            return choose_high_melody(time, high_spectrums2, "red")


high_sequencer = Sequencer(len(high_times)) \
    .add_step_handler(high_step_handler) \
    .start_time_handler(lambda time: time + random_range(3, 5)) \
    .next_time_handler(lambda i: high_times[i] * random_range(0.85, 1.15)) \
    .spawn_sequencer(1, middle_high_sequencer)

low_times = [13, 13, 13, 13, 13, 13, 13, 13]
low_spectrums = [1, 1, 1, 1, 1, 1, 1, 1]

def low_step_handler(i: int, time: float) -> list[SequenceNote]:
    match low_spectrums[i]:
        case 1:
            return choose_low_melody(time, low_spectrums1, "black")
        case 2:
            return choose_low_melody(time, low_spectrums2, "red")


low_sequencer = Sequencer(len(low_times)) \
    .add_step_handler(low_step_handler) \
    .start_time_handler(lambda time: time + random_range(2, 3)) \
    .next_time_handler(lambda i: low_times[i] * random_range(0.85, 1.15)) \
    .spawn_sequencer(1, high_sequencer)

middle_times = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
middle_spectrums = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

def middle_step_handler(i: int, time: float) -> list[SequenceNote]:
    match middle_spectrums[i]:
        case 1:
            return choose_middle_melody(time, middle_spectrums1, "black")
        case 2:
            return choose_middle_melody(time, middle_spectrums2, "red")

sequence_notes = Sequencer(len(middle_times)) \
    .add_step_handler(middle_step_handler) \
    .next_time_handler(lambda i: middle_times[i] * random_range(0.85, 1.15)) \
    .spawn_sequencer(1, low_sequencer) \
    .generate(0)

#sequence_notes = []
ui_piece = UiPieceBuilder() \
    .add_notes(sequence_notes) \
    .build()

piece_duration = ui_piece.get_duration()

TRACK_HEIGHT = 200
NOTE_SCALE_FACTOR = 10
HEIGHT_INDENT = 180

ui_width = (200 + (piece_duration * NOTE_SCALE_FACTOR))
ui_height = (200 * len(ui_piece.tracks))

canvas = Canvas(width=ui_width, height=ui_height)

for track_index, track in enumerate(ui_piece.tracks):
    canvas.font = "18px sans-serif"
    canvas.fill_text(track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT)
    for note in track.notes:
        startx = 200 + (note.start * NOTE_SCALE_FACTOR)
        starty = (track_index * TRACK_HEIGHT) - (note.note * 5) + HEIGHT_INDENT
        peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
        peaky = (track_index * TRACK_HEIGHT) - (note.note * 5) + HEIGHT_INDENT - 5
        endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
        endy = (track_index * TRACK_HEIGHT) - (note.note * 5) + HEIGHT_INDENT
        canvas.stroke_style = note.color    
        canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])
        

canvas


Canvas(height=800, width=1332)